# Adversarial Attacks

### Wczytanie zbioru danych

In [ ]:
from IPython import display
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Compose, Normalize, Resize
from torchvision.models import resnet18
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(1337) 

batch_size = 64 
transforms = Compose([Resize((224, 224)),
                      ToTensor(), 
                      Normalize((0.1307,), (0.3081,))])

# Mnist dataset
train_data = MNIST(root='./data/', 
                   train=True, 
                   transform=transforms,    
                   download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

val_data = MNIST(root='./data/', 
                 train=False, 
                 transform=transforms, 
                 download=True)
val_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)

### Transfer Learning

**Zadanie 1.** Proszę wymienić warstwę wejściową i wyjściową wczytanej sieci ResNet. 

- Wejście powinno mieć 1 kanał wejściowy zamiast 3,
- wyjście powinno być 10-klasowe.

*Hint:* Nazwy warstw można otrzymać wypisując `model`.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = resnet18(pretrained=True)
model.fc = ???
model.conv1 = ???
model.to(device)

In [ ]:
epochs = 1

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

if torch.cuda.is_available():
    torch.cuda.empty_cache()

for epoch in range(1, epochs + 1):
    total_loss = 0
    
    model.train()
    
    progress = tqdm(enumerate(train_loader, 1), desc="Loss: ", total=len(train_loader))
    
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        model.zero_grad()
        outputs = model(X)
        loss = loss_function(outputs, y)
        loss.backward()
        optimizer.step()

        current_loss = loss.item()
        total_loss += current_loss
        
        progress.set_description("Loss: {:.4f}".format(total_loss/(i)))
    
    torch.save(model.state_dict(), 'resnet-mnist.pth')
    
    val_losses = 0
    correct = 0
    num_samples = 0

    model.eval()
    with torch.no_grad():
        for i, data in tqdm(enumerate(val_loader, 1), total=len(val_loader)):
            X, y = data[0].to(device), data[1].to(device)
            outputs = model(X)
            val_losses += loss_function(outputs, y)
            predicted_classes = torch.max(outputs, 1)[1]
            correct += torch.sum(predicted_classes == y).cpu().numpy()
            num_samples += len(y)

    print(f'Validation Accuracy: {correct / num_samples * 100:.2f}%')
    

### Wczytanie zapisanej sieci

In [ ]:
model = resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 10)
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.load_state_dict(torch.load('resnet-mnist.pth'))
model.to(device)

### Atak na Sieć Neuronową

![zwodnicza panda](utils/fgsm_panda_image.png)

Okazuje się, że sieci konwolucyjne można łatwo zmylić wprowadzając szum niewidoczny dla ludzkiego oka. Jedną z metod wprowadzenia takiego szumu jest policzenie gradientów funkcji likelihood po pikselach obrazu wejściowego dla nauczonej wcześniej sieci, a następnie dodanie do obrazka niewielkie, $\epsilon$-owe zaburzenia zgodne ze znakiem gradientu.

Mając więc obraz $x$, wybieramy dostatecznie małe $\epsilon$ i obliczamy gradient funkcji kosztu $J$ dla ustalonych wag $\theta$ przy danej etykiecie $y$, a następnie dodajemy jego znak do oryginalnego obrazu:

$$
    x + \epsilon \cdot \operatorname{sign}(\nabla_x J(\theta, x, y)).
$$

**Zadanie 2.** Uzupełnić kod ataku w zaznaczonych miejscach.

In [ ]:
def fgsm_attack(image, epsilon, data_grad):
    # Pobranie dla każdego piksela znaku gradientu
    sign_data_grad = ???
    # Stworzenie zaburzonego obrazu zgodnie ze wzorem przy użyciu znaków gradientów
    perturbed_image = ???
    return perturbed_image

In [ ]:
test_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=1, shuffle=True)
num_tested = 500
epsilons = [0, .001, 0.003, 0.005, 0.01, 0.1]

model.eval()
examples = []

for epsilon in epsilons:
    correct = 0
    adv_examples = []

    for i, test_data in tqdm(enumerate(test_loader, 1), total=num_tested):
        data, target = test_data[0].to(device), test_data[1].to(device)

        # Ustawienie obliczeń gradientu na obrazie wejściowym
        ???

        # Przejście w przód 
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1]

        # Jeżeli sieć się pomyliła, niepotrzebny jest atak
        if init_pred.item() != target.item():
            continue

        # Obliczenie funkcji kosztu (negative log likelihood)
        loss = ???

        model.zero_grad()
        loss.backward()

        # Zebranie gradientów z obrazu
        data_grad = ???

        # Wywołanie funkcji ataku FGSM
        perturbed_data = ???

        # Ponowna klasyfikacja zniekształconego obrazu
        output = model(perturbed_data)

        final_pred = output.max(1, keepdim=True)[1]
        
        if final_pred.item() == target.item():
            correct += 1
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:
            # Zapisanie obrazów na później
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        
        if i >= num_tested:
            break

    final_acc = correct/float(num_tested)
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, num_tested, final_acc))
    
    examples.append(adv_examples)

In [ ]:
cnt = 0
plt.figure(figsize=(8,10))
for i in range(len(epsilons)):
    for j in range(len(examples[i])):
        cnt += 1
        plt.subplot(len(epsilons),len(examples[0]),cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
        orig,adv,ex = examples[i][j]
        plt.title("{} -> {}".format(orig, adv))
        plt.imshow(ex, cmap="gray")
plt.tight_layout()
plt.show()